# Create lightning stats for June, July and August 2024 
→ Find days for testset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import io
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))
import folium
import glob
import requests
import math

import src.global_variables as global_vars
import src.local_variables as local_vars

In [ ]:
radar = global_vars.RADARS["hurum"]
radar

**Define grid** 
  
Using same strategy as Rombeek et. al. 2024:  
>The point data were transformed to a **gridded binary map**, with 1 indicating lightning activity within a **radius of 8 km** and in the **last 10 min** and 0 otherwise. This definition is used in safety procedures at airports for takeoff and landing operations and based on the **regulations of the European Union** (2017) and International Civil Aviation Organization (2018). In this way, the result of our machine learning algorithm can be directly applied for METAR trend reports without any adjustment of the temporal and spatial resolution.

In [ ]:
# Define 10 x 10 test grid with 8 km spacing
radar_lat = radar["lat"]
radar_lon = radar["lon"]

# Grid parameters
cell_size = 8 # km
n_cells = 10   # number of grid points along one axis
grid_length = cell_size * n_cells # size of grid in km

# Create Cartesian coordinates (in km)
x_bounds_km = np.linspace(-grid_length/2, grid_length/2, n_cells+1)
y_bounds_km = np.linspace(-grid_length/2, grid_length/2, n_cells+1)
x_centers_km = (x_bounds_km[:-1] + x_bounds_km[1:]) / 2
y_centers_km = (y_bounds_km[:-1] + y_bounds_km[1:]) / 2

# Convert Cartesian to lat/lon coordinates
def cartesian_to_latlon(x_km, y_km, radar_lat, radar_lon):
    """Convert Cartesian coordinates to lat/lon"""
    deg2km = 111.0
    lons = radar_lon + x_km / deg2km / np.cos(np.radians(radar_lat))
    lats = radar_lat + y_km / deg2km
    return lats, lons

grid_lats, grid_lons = cartesian_to_latlon(x_bounds_km, y_bounds_km, radar_lat, radar_lon)

# Derive step size
lat_step = grid_lats[1] - grid_lats[0]
lon_step = grid_lons[1] - grid_lons[0]


# Create grid coordinates
deg2km = 111.0 # Approximate conversion from degrees to kilometers (earth circumference / 360 degrees)
lat_step = cell_size/deg2km
lon_step = cell_size/(deg2km * np.cos(np.radians(radar_lat)))

# Save the grid parameters to a .npz file for later use
# np.savez('../data/radar_hurum_grid_10x10_8km_spacing.npz',
#          radar_lat=radar_lat,
#          radar_lon=radar_lon,
#          cell_size=cell_size,
#          n_cells=n_cells,
#          grid_length=grid_length,
#          lat_step=lat_step,
#          lon_step=lon_step,
#          grid_lats=grid_lats,
#          grid_lons=grid_lons,
#          x_bounds_km=x_bounds_km,
#          y_bounds_km=y_bounds_km,
#          x_centers_km=x_centers_km,
#          y_centers_km=y_centers_km)

In [ ]:
# Visualize grid
grid = folium.Map(location=(radar_lat, radar_lon), zoom_start=9)

# Add grid lines
for i in range(n_cells + 1):
    # Horizontal lines
    folium.PolyLine(
        locations=[[grid_lats[i], grid_lons[0]], [grid_lats[i], grid_lons[-1]]],
        color='black',
        weight=1,
        opacity=0.7
    ).add_to(grid)
    
    # Vertical lines  
    folium.PolyLine(
        locations=[[grid_lats[0], grid_lons[i]], [grid_lats[-1], grid_lons[i]]],
        color='black',
        weight=1,
        opacity=0.7
    ).add_to(grid)

# Add radar location
folium.Marker(
    location=[radar_lat, radar_lon],
    popup='Hurum radar',
    icon=folium.Icon(color='green', icon='tower')
).add_to(grid)

grid

**Read lightning data from Frost**

In [ ]:
# Create box around radar defined by the grid
min_lat = np.min(grid_lats)
max_lat = np.max(grid_lats)
min_lon = np.min(grid_lons)
max_lon = np.max(grid_lons)

polygon = f"POLYGON(({min_lon} {min_lat},{max_lon} {min_lat},{max_lon} {max_lat},{min_lon} {max_lat},{min_lon} {min_lat}))"


In [ ]:
# If lightning data already exists, read it from file. Else, fetch it from the FROST API in separate calls for each month.

filename = '../data/lightning_data.csv'

if os.path.exists(filename):
    print("Loading lightning data from file...")
    lightning_data = pd.read_csv(filename)
    print(f"Loaded {len(lightning_data)} lightning strikes from file.")
else:
    print("File not found. Fetching data from FROST API...")
    client_id = local_vars.FROST_CLIENT_ID
    endpoint = global_vars.FROST_ENDPOINT

    time_ranges = [
        '2024-05-01T00:00:00Z/2024-05-31T23:59:59Z', # May 2024
        '2024-06-01T00:00:00Z/2024-06-30T23:59:59Z', # June 2024
        '2024-07-01T00:00:00Z/2024-07-31T23:59:59Z', # July 2024
        '2024-08-01T00:00:00Z/2024-08-31T23:59:59Z', # August 2024
    ]

    columns = ['year', 'month', 'day', 'hour', 'minute', 'second', 'nanoseconds',
            'lat', 'lon', 'peak_current', 'multi', 'nsens', 'dof', 'angle', 
            'major', 'minor', 'chi2', 'rt', 'ptz', 'mrr', 'cloud', 'aI', 'sI', 'tI']


    all_data = []
    for time_range in time_ranges:
        params = {
            'referencetime': time_range,
            'geometry': polygon
        }
        
        data = requests.get(endpoint, params=params, auth=(client_id, ''))
        
        df_temp = pd.read_csv(io.StringIO(data.text), sep=' ', names=columns)
        df_temp.reset_index()
        all_data.append(df_temp)

    lightning_data = pd.concat(all_data, ignore_index=True)  
    lightning_data.to_csv(filename, index=False)
    print(f"Saved {len(lightning_data)} lightning strikes to file")

In [ ]:
# Plot lightning strikes on a map
lightning_map_all = folium.Map(location=(radar['lat'], radar['lon']), zoom_start=9)

for _, strike in lightning_data.iterrows():
    day = strike['day']
    color = 'red' if strike['cloud'] == 0 else 'blue'
    radius = 1
    folium.CircleMarker(
        location=[strike['lat'], strike['lon']],
        color=color,
        radius=radius,
        fillColor=color,
        fillOpacity=1
        ).add_to(lightning_map_all)
    
# Add radar location
folium.Marker(
    location=[radar_lat, radar_lon],
    popup='Hurum radar',
    icon=folium.Icon(color='green', icon='tower')
).add_to(lightning_map_all)
    
lightning_map_all

**Create overview of lightning counter per day**

In [ ]:
# Convert year, month and day to datetime and add a date column
lightning_data['datetime'] = pd.to_datetime(lightning_data[['year', 'month', 'day']])
lightning_data['date'] = lightning_data['datetime'].dt.date

In [ ]:
# Plot the daily lightning activity for the summer months of 2024 (May to August)
daily_counts = lightning_data.groupby(['date', 'cloud']).size().reset_index(name='count')
intracloud_strikes = daily_counts[daily_counts['cloud']==1]
ground_strikes = daily_counts[daily_counts['cloud']==0]

all_dates = pd.date_range(start=daily_counts['date'].min(), end=daily_counts['date'].max(), freq='D')
ground_counts = all_dates.map(ground_strikes.set_index('date')['count']).fillna(0)
intracloud_counts = all_dates.map(intracloud_strikes.set_index('date')['count']).fillna(0)

plt.figure(figsize=(16, 6))
plt.bar(all_dates, ground_counts, color='#d62728', label='Cloud-to-ground', alpha=0.8)
plt.bar(all_dates, intracloud_counts, bottom=ground_counts, color='#1f77b4', label='Intracloud', alpha=0.8)

plt.title('Daily lightning activity (summer 2024)', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Number of lightning strikes', fontsize=12)
plt.legend(fontsize=11)
plt.xticks(rotation=45) 
plt.xticks(all_dates, rotation=90, fontsize=8)
plt.grid(True, alpha=0.3) 
plt.tight_layout() 
plt.savefig('../outputs/lightning_activity_summer_2024.png', dpi=300)
plt.show()

In [ ]:
intracloud_counts

**Create lightning heatmap for busiest day (1 June 2024)**

In [ ]:
# Find day with most lightning strikes
max_day = lightning_data.groupby('date').size().idxmax()
max_day_data = lightning_data[lightning_data['date']==max_day]

max_day_map = folium.Map(location=(radar['lat'], radar['lon']), zoom_start=9)

# Define grid and draw heatmap rectangles
for i in range(n_cells): # Loop over latitudes
    for j in range(n_cells): # Loop over longitudes
        lat_min = grid_lats[i]
        lat_max = grid_lats[i+1]
        lon_min = grid_lons[j]
        lon_max = grid_lons[j+1]

        # Filter out strikes within grid cell
        strikes_in_cell = max_day_data[
            max_day_data['lat'].between(lat_min, lat_max) & 
            max_day_data['lon'].between(lon_min, lon_max)
        ]

        n_strikes = len(strikes_in_cell)

        if n_strikes > 0:
            if n_strikes <= 5:
                color = 'yellow'
                opacity = 0.4
            elif n_strikes <= 15:
                color = 'gold'
                opacity = 0.6
            elif n_strikes <= 50:
                color = 'orange'
                opacity = 0.6
            else:
                color = 'red'
                opacity = 0.8

            folium.Rectangle(
                bounds=[[lat_min, lon_min], [lat_max, lon_max]],
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=opacity,
                popup=f"{n_strikes} strikes",
                weight=1
            ).add_to(max_day_map)

# Add grid lines
for i in range(n_cells + 1):
    # Horizontal lines
    folium.PolyLine(
        locations=[[grid_lats[i], grid_lons[0]], [grid_lats[i], grid_lons[-1]]],
        color='black',
        weight=1,
        opacity=0.7
    ).add_to(max_day_map)
    
    # Vertical lines  
    folium.PolyLine(
        locations=[[grid_lats[0], grid_lons[i]], [grid_lats[-1], grid_lons[i]]],
        color='black',
        weight=1,
        opacity=0.7
    ).add_to(max_day_map)
            
for _, strike in max_day_data.iterrows():
    color = 'darkred' if strike['cloud'] == 0 else 'darkblue'
    
    folium.CircleMarker(
        location=[strike['lat'], strike['lon']],
        radius=2,  
        color=color,
        fillColor=color,
        fillOpacity=0.8,
        weight=1,
    ).add_to(max_day_map)

# Add radar location
folium.Marker(
    location=[radar_lat, radar_lon],
    popup='Hurum radar',
    icon=folium.Icon(color='green', icon='tower')
).add_to(max_day_map)

max_day_map.save('../outputs/lightning_max_day_map.html')
max_day_map

**Create hourly lightning count of busiest day (1 June 2024)**

In [ ]:
# Plot the hourly lightning activity for the busiest day (1 June 2024)
hourly_counts = max_day_data.groupby(['hour', 'cloud']).size().reset_index(name='count')
intracloud_strikes = hourly_counts[hourly_counts['cloud']==1]
ground_strikes = hourly_counts[hourly_counts['cloud']==0]

# Create all hours for the day
all_hours = range(0, 24)
ground_counts = pd.Series(all_hours).map(ground_strikes.set_index('hour')['count']).fillna(0)
intracloud_counts = pd.Series(all_hours).map(intracloud_strikes.set_index('hour')['count']).fillna(0)

plt.figure(figsize=(16, 6))
plt.bar(all_hours, ground_counts, color='#d62728', label='Cloud-to-ground', alpha=0.8)
plt.bar(all_hours, intracloud_counts, bottom=ground_counts, color='#1f77b4', label='Intracloud', alpha=0.8)

plt.title('Hourly lightning activity (1 June 2024)', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Number of lightning strikes', fontsize=12)
plt.legend(fontsize=11)
plt.xticks(rotation=45) 
plt.xticks(all_hours, rotation=90, fontsize=8)
plt.grid(True, alpha=0.3) 
plt.tight_layout() 
plt.savefig('../outputs/lightning_activity_01-06-2024.png', dpi=300)
plt.show()

In [ ]:
# Plot lightnings per hour for all lightning days in 2024
lightnings_per_day = lightning_data.groupby('date')

for day, df in lightnings_per_day:
    hourly_counts = df.groupby(['hour', 'cloud']).size().reset_index(name='count')
    intracloud_strikes = hourly_counts[hourly_counts['cloud']==1]
    ground_strikes = hourly_counts[hourly_counts['cloud']==0]
    
    # Create all hours for the day
    all_hours = range(0, 24)
    ground_counts = pd.Series(all_hours).map(ground_strikes.set_index('hour')['count']).fillna(0)
    intracloud_counts = pd.Series(all_hours).map(intracloud_strikes.set_index('hour')['count']).fillna(0)

    plt.figure(figsize=(16, 6))
    plt.bar(all_hours, ground_counts, color='#d62728', label='Cloud-to-ground', alpha=0.8)
    plt.bar(all_hours, intracloud_counts, bottom=ground_counts, color='#1f77b4', label='Intracloud', alpha=0.8)

    plt.title(f'Hourly lightning activity ({day})', fontsize=16, fontweight='bold')
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Number of lightning strikes', fontsize=12)
    plt.legend(fontsize=11)
    plt.xticks(rotation=45) 
    plt.xticks(all_hours, rotation=90, fontsize=8)
    plt.grid(True, alpha=0.3) 
    plt.tight_layout() 
    plt.savefig(f'../outputs/lightning_days_2024/lightning_activity_{day}.png', dpi=300)
    plt.close();

In [ ]:
# Create file for storm periods
# filename = "storm_periods.csv"

# with open(f"../data/{filename}", "w") as f:
#     f.write("date,start_time,end_time")
#     for day, _ in lightnings_per_day:
#         f.write(f"\n{day},")

**Plot hourly evolution of storm as a heatmap**

In [ ]:
all_matrices = []
time_step = 5 # minutes

for hour in range(24):
    for minute in range(0, 60, time_step):
        grid_counts = np.zeros((n_cells, n_cells))
        time_slice = max_day_data[(max_day_data['hour']==hour) & 
                                  (max_day_data['minute']>=minute) & 
                                  (max_day_data['minute']<minute+time_step)]
        
        for _, strike in time_slice.iterrows():
            lat_idx = np.digitize(strike['lat'], grid_lats) - 1
            lon_idx = np.digitize(strike['lon'], grid_lons) - 1

            if 0 <= lat_idx < n_cells and 0 <= lon_idx < n_cells:
                grid_counts[lat_idx, lon_idx] += 1

        all_matrices.append(grid_counts)

max_value = max([np.max(matrix) for matrix in all_matrices])

# Plot all heatmaps for each time step

# Loop over all matrices and plot them
for i, matrix in enumerate(all_matrices):
    hour = int(i // (60/time_step))
    minute = int((i % (60/time_step)) * time_step)
    
    if (hour>=10) and (hour<16):
        plt.figure(figsize=(8, 8))
        plt.imshow(matrix, cmap='YlOrRd', origin='lower', 
                vmin=0, vmax=max_value, interpolation='nearest')
        
        plt.colorbar(label='Lightning strikes')
        plt.title(f'Lightning activity - 1 June 2024 @ {hour:02d}:{minute:02d}\nTotal strikes: {int(matrix.sum())}')
        plt.xlabel('Grid cell (longitude)')
        plt.ylabel('Grid cell (latitude)')

        # Add grid lines
        for i in range(n_cells + 1):
            plt.axhline(i - 0.5, color='black', linewidth=0.5, alpha=0.3)
            plt.axvline(i - 0.5, color='black', linewidth=0.5, alpha=0.3)
    
        plt.tight_layout()
        plt.savefig(f'../outputs/01-06-2024-storm/lightning_heatmap_{hour:02d}_{minute:02d}.png', dpi=300)
        plt.close()